In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import cv2

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
df = pd.read_csv("/kaggle/input/attention-cnn-model/patient_diagnosis.csv")
df.head()

,101,URTI
0,102,Healthy
1,103,Asthma
2,104,COPD
3,105,URTI
4,106,COPD


In [3]:
df['URTI'].unique()

array(['Healthy', 'Asthma', 'COPD', 'URTI', 'LRTI', 'Bronchiectasis',
       'Pneumonia', 'Bronchiolitis'], dtype=object)

In [4]:
sr_no = {'101':'URTI'}
for i, j in zip(df['101'].unique(), df['URTI']):
    sr_no[str(i)] = j

In [5]:
sr_no.keys()

dict_keys(['101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226'])

In [6]:
import os
sound_files = os.listdir('/kaggle/input/attention-cnn-model/Mel Spectrogram/Mel Spectrogram/Original')

In [7]:
req_file_names = []

for i in sound_files:
      req_file_names.append([i])

req_file_names

[['176_2b3_Lr_mc_AKGC417L.png'],
 ['112_1p1_Pr_sc_Litt3200.png'],
 ['130_2b2_Pr_mc_AKGC417L.png'],
 ['193_7b3_Ll_mc_AKGC417L.png'],
 ['156_5b3_Pl_mc_AKGC417L.png'],
 ['162_2b2_Tc_mc_AKGC417L.png'],
 ['134_2b3_Ar_mc_LittC2SE.png'],
 ['160_1b4_Tc_mc_AKGC417L.png'],
 ['200_2p4_Pl_mc_AKGC417L.png'],
 ['169_1b2_Ll_sc_Meditron.png'],
 ['144_1b1_Tc_sc_Meditron.png'],
 ['160_2b4_Pl_mc_AKGC417L.png'],
 ['133_2p3_Pr_mc_AKGC417L.png'],
 ['205_1b3_Al_mc_AKGC417L.png'],
 ['130_3p4_Tc_mc_AKGC417L.png'],
 ['130_1p4_Lr_mc_AKGC417L.png'],
 ['200_3p4_Pl_mc_AKGC417L.png'],
 ['107_2b4_Ar_mc_AKGC417L.png'],
 ['203_2p3_Al_mc_AKGC417L.png'],
 ['113_1b1_Ar_sc_Litt3200.png'],
 ['174_2p3_Al_mc_AKGC417L.png'],
 ['178_1b2_Lr_mc_AKGC417L.png'],
 ['172_1b5_Ll_mc_AKGC417L.png'],
 ['216_1b1_Al_sc_Meditron.png'],
 ['205_4b2_Pl_mc_AKGC417L.png'],
 ['177_1b2_Lr_mc_AKGC417L.png'],
 ['151_2p4_Ll_mc_AKGC417L.png'],
 ['141_1b1_Pr_mc_LittC2SE.png'],
 ['162_1b2_Lr_mc_AKGC417L.png'],
 ['133_3p4_Tc_mc_AKGC417L.png'],
 ['147_2b3

In [8]:
labels = []
for i in range(len(req_file_names)):
    req_file_names[i].append(sr_no[req_file_names[i][0][:3]])
    labels.append(sr_no[req_file_names[i][0][:3]])

In [9]:
labels *= 3

In [10]:
x = []

for i in req_file_names:
    img = cv2.imread('/kaggle/input/attention-cnn-model/Mel Spectrogram/Mel Spectrogram/Time Stretch/'+i[0])
    img = cv2.resize(img, (350, 350))
    x.append(img)

for i in req_file_names:
    img = cv2.imread('/kaggle/input/attention-cnn-model/Mel Spectrogram/Mel Spectrogram/Pitch Shift/'+i[0])
    img = cv2.resize(img, (350, 350))
    x.append(img)
    
for i in req_file_names:
    img = cv2.imread('/kaggle/input/attention-cnn-model/Mel Spectrogram/Mel Spectrogram/Audio Shift/'+i[0])
    img = cv2.resize(img, (350, 350))
    x.append(img)

# x = np.array(x)
# print(x.shape)

In [11]:
x_new = []
y = []

for i in range(len(labels)):
    if labels[i]=='Asthma' or labels[i]=='Bronchiolitis':
        continue
    x_new.append(x[i])
    y.append(labels[i])

In [12]:
x = np.array(x_new)
print(x.shape)

(2718, 350, 350, 3)


In [13]:
req_file_names *= 3

In [14]:
y = np.array(y)
y.shape

(2718,)

In [15]:
one_hot_y = np.array(pd.get_dummies(y))

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, one_hot_y, test_size=0.2, random_state=39, stratify=y)
print(x_train.shape, y_train.shape, x_test.shape)

(2174, 350, 350, 3) (2174, 6) (544, 350, 350, 3)


In [17]:
checkpoint_cnn = keras.callbacks.ModelCheckpoint("/kaggle/working/CNN_model-{epoch:02d}.h5", save_best_only = True)

In [18]:
input1 = keras.layers.Input(shape=(350, 350, 3))
conv1 = keras.layers.Conv2D(64, (7,7), activation='relu')(input1)
mp1 = keras.layers.MaxPool2D((2,2))(conv1)
conv2 = keras.layers.Conv2D(128, (5,5), activation='relu')(mp1)
mp2 = keras.layers.MaxPool2D((2,2))(conv2)
do1 = keras.layers.Dropout(0.2)(mp2)
conv3 = keras.layers.Conv2D(256, (3,3), activation='relu')(do1)
mp3 = keras.layers.MaxPool2D((2,2))(conv3)

# random = np.random.random((mp3.shape[1:]))

attention = keras.layers.Attention()([mp3, mp3])

flatten = keras.layers.Flatten()(attention)
dense1 = keras.layers.Dense(128, activation='relu')(flatten)
bn = keras.layers.BatchNormalization()(dense1)
dense2 = keras.layers.Dense(64, activation='relu')(bn)
output = keras.layers.Dense(6, activation='softmax')(dense2)

model = keras.Model(inputs=input1, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 344, 344, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 172, 172, 64  0           ['conv2d[0][0]']                 
                                )                                                             

In [19]:
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
history_cnn = model.fit(x_train, y_train, batch_size=32, epochs=120, validation_data=(x_test, y_test))

Epoch 1/120


2023-08-16 11:22:57.582382: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


68/68 [==============================] - 36s 340ms/step - loss: 2.4795 - accuracy: 0.1150 - val_loss: 1.2546 - val_accuracy: 0.7132
Epoch 2/120
68/68 [==============================] - 18s 264ms/step - loss: 1.7493 - accuracy: 0.3712 - val_loss: 9.0239 - val_accuracy: 0.0386
Epoch 3/120
68/68 [==============================] - 18s 263ms/step - loss: 1.3458 - accuracy: 0.5534 - val_loss: 3.3834 - val_accuracy: 0.0110
Epoch 4/120
68/68 [==============================] - 18s 263ms/step - loss: 1.1325 - accuracy: 0.6886 - val_loss: 2.3341 - val_accuracy: 0.0919
Epoch 5/120
68/68 [==============================] - 17s 255ms/step - loss: 0.9968 - accuracy: 0.7539 - val_loss: 1.8681 - val_accuracy: 0.1857
Epoch 6/120
68/68 [==============================] - 19s 287ms/step - loss: 0.9249 - accuracy: 0.7944 - val_loss: 0.8394 - val_accuracy: 0.7592
Epoch 7/120
68/68 [==============================] - 18s 264ms/step - loss: 0.8654 - accuracy: 0.8063 - val_loss: 0.9946 - val_accuracy: 0.7224
Epoc

In [21]:
df = pd.DataFrame(history_cnn.history)
df.to_csv('/kaggle/working/CNN_history.csv')

# Transfer Learning

### Xception model

In [22]:
xception_wo_top = keras.applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

83683744/83683744 [==============================] - 3s 0us/step


In [23]:
xception_wo_top.trainable = False

In [24]:
input_xception = keras.layers.Input(shape=(350, 350, 3))
xception_layer = xception_wo_top(input_xception)

# random = np.random.random(xception_layer.shape[1:])

attention = keras.layers.Attention()([xception_layer, xception_layer])

flatten = keras.layers.Flatten()(attention)

dense1 = keras.layers.Dense(128, activation='relu')(flatten)
bn1 = keras.layers.BatchNormalization()(dense1)
dense2 = keras.layers.Dense(64, activation='relu')(bn1)
output = keras.layers.Dense(6, activation='softmax')(dense2)

xception_model = keras.Model(inputs=input_xception, outputs=output)
xception_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 xception (Functional)          (None, 11, 11, 2048  20861480    ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 attention_1 (Attention)        (None, 11, 11, 2048  0           ['xception[0][0]',               
                                )                                 'xception[0][0]']         

In [25]:
checkpoint_xception = keras.callbacks.ModelCheckpoint('/kaggle/working/Xception_Model.h5', save_best_only=True)

In [26]:
xception_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
history_xception = xception_model.fit(x_train, y_train, batch_size=32, epochs=120, validation_data=(x_test, y_test), callbacks=[checkpoint_xception])

Epoch 1/120
68/68 [==============================] - 27s 311ms/step - loss: 0.7140 - accuracy: 0.8119 - val_loss: 0.5357 - val_accuracy: 0.8750
Epoch 2/120
68/68 [==============================] - 18s 273ms/step - loss: 0.4189 - accuracy: 0.8855 - val_loss: 0.3988 - val_accuracy: 0.8842
Epoch 3/120
68/68 [==============================] - 16s 238ms/step - loss: 0.3461 - accuracy: 0.9103 - val_loss: 0.3546 - val_accuracy: 0.8934
Epoch 4/120
68/68 [==============================] - 17s 249ms/step - loss: 0.2932 - accuracy: 0.9264 - val_loss: 0.3551 - val_accuracy: 0.9044
Epoch 5/120
68/68 [==============================] - 15s 214ms/step - loss: 0.2529 - accuracy: 0.9480 - val_loss: 0.3622 - val_accuracy: 0.9081
Epoch 6/120
68/68 [==============================] - 16s 236ms/step - loss: 0.2092 - accuracy: 0.9660 - val_loss: 0.3095 - val_accuracy: 0.9173
Epoch 7/120
68/68 [==============================] - 16s 235ms/step - loss: 0.1737 - accuracy: 0.9770 - val_loss: 0.2903 - val_accuracy:

In [28]:
df = pd.DataFrame(history_xception.history)
df.to_csv('/kaggle/working/Xception_history.csv')

### VGG19

In [29]:
vgg_wo_top = keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

80134624/80134624 [==============================] - 3s 0us/step


In [30]:
vgg_wo_top.trainable = False

In [31]:
input_vgg = keras.layers.Input(shape=(350, 350, 3))
vgg_layer = vgg_wo_top(input_vgg)

# random = np.random.random(vgg_layer.shape[1:])

attention = keras.layers.Attention()([vgg_layer, vgg_layer])
flatten = keras.layers.Flatten()(attention)

dense1 = keras.layers.Dense(128, activation='relu')(flatten)
bn1 = keras.layers.BatchNormalization()(dense1)
dense2 = keras.layers.Dense(64, activation='relu')(bn1)
output = keras.layers.Dense(6, activation='softmax')(dense2)

vgg_model = keras.Model(inputs=input_vgg, outputs=output)
vgg_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg19 (Functional)             (None, 10, 10, 512)  20024384    ['input_5[0][0]']                
                                                                                                  
 attention_2 (Attention)        (None, 10, 10, 512)  0           ['vgg19[0][0]',                  
                                                                  'vgg19[0][0]']                  
                                                                                            

In [32]:
checkpoint_vgg = keras.callbacks.ModelCheckpoint('/kaggle/working/VGG_Model.h5', save_best_only=True)

In [33]:
vgg_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
history_vgg = vgg_model.fit(x_train, y_train, batch_size=32, epochs=120, validation_data=(x_test, y_test), callbacks=[checkpoint_vgg])

Epoch 1/120
68/68 [==============================] - 28s 299ms/step - loss: 1.5538 - accuracy: 0.3910 - val_loss: 1.8324 - val_accuracy: 0.3695
Epoch 2/120
68/68 [==============================] - 16s 232ms/step - loss: 0.8592 - accuracy: 0.7691 - val_loss: 0.7530 - val_accuracy: 0.7390
Epoch 3/120
68/68 [==============================] - 18s 263ms/step - loss: 0.6421 - accuracy: 0.8721 - val_loss: 0.4914 - val_accuracy: 0.8695
Epoch 4/120
68/68 [==============================] - 16s 231ms/step - loss: 0.5210 - accuracy: 0.9006 - val_loss: 0.4650 - val_accuracy: 0.8879
Epoch 5/120
68/68 [==============================] - 16s 230ms/step - loss: 0.4512 - accuracy: 0.9195 - val_loss: 0.4063 - val_accuracy: 0.8952
Epoch 6/120
68/68 [==============================] - 16s 230ms/step - loss: 0.3892 - accuracy: 0.9434 - val_loss: 0.3617 - val_accuracy: 0.9265
Epoch 7/120
68/68 [==============================] - 15s 225ms/step - loss: 0.3501 - accuracy: 0.9535 - val_loss: 0.3897 - val_accuracy:

In [35]:
df = pd.DataFrame(history_vgg.history)
df.to_csv('/kaggle/working/VGG_history.csv')

### EfficientNet B0

In [36]:
enb0_wo_top = keras.applications.efficientnet.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb0_wo_top.trainable = False

input_enb0 = keras.layers.Input(shape=(350, 350, 3))
enb0_layer = enb0_wo_top(input_enb0)

# random = np.random.random(enb0_layer.shape[1:])

attention = keras.layers.Attention()([enb0_layer, enb0_layer])
flatten = keras.layers.Flatten()(attention)

dense1 = keras.layers.Dense(128, activation='relu')(flatten)
bn1 = keras.layers.BatchNormalization()(dense1)
dense2 = keras.layers.Dense(64, activation='relu')(bn1)
output = keras.layers.Dense(6, activation='softmax')(dense2)

enb0_model = keras.Model(inputs=input_enb0, outputs=output)
enb0_model.summary()

16705208/16705208 [==============================] - 1s 0us/step
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnetb0 (Functional)    (None, 11, 11, 1280  4049571     ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 attention_3 (Attention)        (None, 11, 11, 1280  0           ['efficientnetb0[0][0]',         
                           

In [37]:
checkpoint_enb0 = keras.callbacks.ModelCheckpoint('/kaggle/working/ENB0_Model.h5', save_best_only=True)

enb0_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
history_enb0 = enb0_model.fit(x_train, y_train, batch_size=32, epochs=120, validation_data=(x_test, y_test))

Epoch 1/120


2023-08-16 13:05:20.786601: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


68/68 [==============================] - 19s 157ms/step - loss: 0.8154 - accuracy: 0.7764 - val_loss: 0.8499 - val_accuracy: 0.7371
Epoch 2/120
68/68 [==============================] - 9s 128ms/step - loss: 0.4403 - accuracy: 0.9062 - val_loss: 0.4925 - val_accuracy: 0.8860
Epoch 3/120
68/68 [==============================] - 9s 129ms/step - loss: 0.3738 - accuracy: 0.9218 - val_loss: 0.4444 - val_accuracy: 0.8934
Epoch 4/120
68/68 [==============================] - 8s 111ms/step - loss: 0.3339 - accuracy: 0.9315 - val_loss: 0.3648 - val_accuracy: 0.9246
Epoch 5/120
68/68 [==============================] - 7s 110ms/step - loss: 0.2922 - accuracy: 0.9407 - val_loss: 0.2817 - val_accuracy: 0.9522
Epoch 6/120
68/68 [==============================] - 7s 110ms/step - loss: 0.2631 - accuracy: 0.9494 - val_loss: 0.2746 - val_accuracy: 0.9449
Epoch 7/120
68/68 [==============================] - 7s 111ms/step - loss: 0.2516 - accuracy: 0.9535 - val_loss: 0.3070 - val_accuracy: 0.9522
Epoch 8/12

In [39]:
df = pd.DataFrame(history_enb0.history)
df.to_csv('/kaggle/working/ENB0_history.csv')

### EfficientNet B1

In [40]:
enb1_wo_top = keras.applications.efficientnet.EfficientNetB1(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb1_wo_top.trainable = False

input_enb1 = keras.layers.Input(shape=(350, 350, 3))
enb1_layer = enb1_wo_top(input_enb1)

# random = np.random.random(enb1_layer.shape[1:])

attention = keras.layers.Attention()([enb1_layer, enb1_layer])
flatten = keras.layers.Flatten()(attention)

dense1 = keras.layers.Dense(128, activation='relu')(flatten)
bn1 = keras.layers.BatchNormalization()(dense1)
dense2 = keras.layers.Dense(64, activation='relu')(bn1)
output = keras.layers.Dense(6, activation='softmax')(dense2)

enb1_model = keras.Model(inputs=input_enb1, outputs=output)
enb1_model.summary()

checkpoint_enb1 = keras.callbacks.ModelCheckpoint('/kaggle/working/ENB1_Model.h5', save_best_only=True)

enb1_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_enb1 = enb1_model.fit(x_train, y_train, batch_size=32, epochs=120, validation_data=(x_test, y_test))

df = pd.DataFrame(history_enb1.history)
df.to_csv('/kaggle/working/ENB1_history.csv')

27018416/27018416 [==============================] - 2s 0us/step
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnetb1 (Functional)    (None, 11, 11, 1280  6575239     ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 attention_4 (Attention)        (None, 11, 11, 1280  0           ['efficientnetb1[0][0]',         
                           

2023-08-16 13:21:32.052864: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/efficientnetb1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


68/68 [==============================] - 24s 199ms/step - loss: 0.7892 - accuracy: 0.7705 - val_loss: 0.5624 - val_accuracy: 0.8603
Epoch 2/120
68/68 [==============================] - 10s 152ms/step - loss: 0.4166 - accuracy: 0.8956 - val_loss: 0.3030 - val_accuracy: 0.9081
Epoch 3/120
68/68 [==============================] - 10s 151ms/step - loss: 0.3203 - accuracy: 0.9177 - val_loss: 0.3200 - val_accuracy: 0.9338
Epoch 4/120
68/68 [==============================] - 10s 151ms/step - loss: 0.2873 - accuracy: 0.9255 - val_loss: 0.2781 - val_accuracy: 0.9357
Epoch 5/120
68/68 [==============================] - 10s 152ms/step - loss: 0.2489 - accuracy: 0.9351 - val_loss: 0.3211 - val_accuracy: 0.9357
Epoch 6/120
68/68 [==============================] - 11s 162ms/step - loss: 0.2264 - accuracy: 0.9499 - val_loss: 0.2416 - val_accuracy: 0.9504
Epoch 7/120
68/68 [==============================] - 10s 152ms/step - loss: 0.2067 - accuracy: 0.9545 - val_loss: 0.3634 - val_accuracy: 0.9357
Epoc

### EfficientNet B2

In [41]:
enb2_wo_top = keras.applications.efficientnet.EfficientNetB2(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb2_wo_top.trainable = False

input_enb2 = keras.layers.Input(shape=(350, 350, 3))
enb2_layer = enb2_wo_top(input_enb2)

# random = np.random.random(enb2_layer.shape[1:])

attention = keras.layers.Attention()([enb2_layer, enb2_layer])
flatten = keras.layers.Flatten()(attention)

dense1 = keras.layers.Dense(128, activation='relu')(flatten)
bn1 = keras.layers.BatchNormalization()(dense1)
dense2 = keras.layers.Dense(64, activation='relu')(bn1)
output = keras.layers.Dense(6, activation='softmax')(dense2)

enb2_model = keras.Model(inputs=input_enb2, outputs=output)
enb2_model.summary()

checkpoint_enb2 = keras.callbacks.ModelCheckpoint('/kaggle/working/ENB2_Model.h5', save_best_only=True)

enb2_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_enb2 = enb2_model.fit(x_train, y_train, batch_size=32, epochs=120, validation_data=(x_test, y_test))

df = pd.DataFrame(history_enb2.history)
df.to_csv('/kaggle/working/ENB2_history.csv')

31790344/31790344 [==============================] - 1s 0us/step
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnetb2 (Functional)    (None, 11, 11, 1408  7768569     ['input_11[0][0]']               
                                )                                                                 
                                                                                                  
 attention_5 (Attention)        (None, 11, 11, 1408  0           ['efficientnetb2[0][0]',         
                           

2023-08-16 13:43:05.092803: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_5/efficientnetb2/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


68/68 [==============================] - 25s 206ms/step - loss: 1.4207 - accuracy: 0.5359 - val_loss: 1.5732 - val_accuracy: 0.2941
Epoch 2/120
68/68 [==============================] - 11s 169ms/step - loss: 0.8868 - accuracy: 0.8694 - val_loss: 1.1791 - val_accuracy: 0.7040
Epoch 3/120
68/68 [==============================] - 11s 161ms/step - loss: 0.7519 - accuracy: 0.8988 - val_loss: 1.1884 - val_accuracy: 0.6820
Epoch 4/120
68/68 [==============================] - 11s 159ms/step - loss: 0.6845 - accuracy: 0.9089 - val_loss: 0.8390 - val_accuracy: 0.8934
Epoch 5/120
68/68 [==============================] - 11s 160ms/step - loss: 0.6339 - accuracy: 0.9223 - val_loss: 0.9462 - val_accuracy: 0.8438
Epoch 6/120
68/68 [==============================] - 12s 170ms/step - loss: 0.5919 - accuracy: 0.9209 - val_loss: 0.7649 - val_accuracy: 0.9246
Epoch 7/120
68/68 [==============================] - 11s 169ms/step - loss: 0.5509 - accuracy: 0.9259 - val_loss: 0.7534 - val_accuracy: 0.9154
Epoc

### EfficientNet B3

In [42]:
enb3_wo_top = keras.applications.efficientnet.EfficientNetB3(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb3_wo_top.trainable = False

input_enb3 = keras.layers.Input(shape=(350, 350, 3))
enb3_layer = enb3_wo_top(input_enb3)

# random = np.random.random(enb3_layer.shape[1:])

attention = keras.layers.Attention()([enb3_layer, enb3_layer])
flatten = keras.layers.Flatten()(attention)

dense1 = keras.layers.Dense(128, activation='relu')(flatten)
bn1 = keras.layers.BatchNormalization()(dense1)
dense2 = keras.layers.Dense(64, activation='relu')(bn1)
output = keras.layers.Dense(6, activation='softmax')(dense2)

enb3_model = keras.Model(inputs=input_enb3, outputs=output)
enb3_model.summary()

checkpoint_enb3 = keras.callbacks.ModelCheckpoint('/kaggle/working/ENB3_Model.h5', save_best_only=True)

enb3_model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_enb3 = enb3_model.fit(x_train, y_train, batch_size=32, epochs=120, validation_data=(x_test, y_test))

df = pd.DataFrame(history_enb3.history)
df.to_csv('/kaggle/working/ENB3_history.csv')

43941136/43941136 [==============================] - 2s 0us/step
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnetb3 (Functional)    (None, 11, 11, 1536  10783535    ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 attention_6 (Attention)        (None, 11, 11, 1536  0           ['efficientnetb3[0][0]',         
                           

2023-08-16 14:05:46.567780: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_6/efficientnetb3/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


68/68 [==============================] - 32s 287ms/step - loss: 0.6563 - accuracy: 0.8243 - val_loss: 0.4646 - val_accuracy: 0.8768
Epoch 2/120
68/68 [==============================] - 14s 206ms/step - loss: 0.3623 - accuracy: 0.9080 - val_loss: 0.3580 - val_accuracy: 0.9154
Epoch 3/120
68/68 [==============================] - 14s 204ms/step - loss: 0.3021 - accuracy: 0.9278 - val_loss: 0.3027 - val_accuracy: 0.9246
Epoch 4/120
68/68 [==============================] - 14s 204ms/step - loss: 0.2750 - accuracy: 0.9328 - val_loss: 0.4061 - val_accuracy: 0.9081
Epoch 5/120
68/68 [==============================] - 14s 205ms/step - loss: 0.2434 - accuracy: 0.9517 - val_loss: 0.2744 - val_accuracy: 0.9540
Epoch 6/120
68/68 [==============================] - 14s 205ms/step - loss: 0.2240 - accuracy: 0.9517 - val_loss: 0.1943 - val_accuracy: 0.9651
Epoch 7/120
68/68 [==============================] - 14s 204ms/step - loss: 0.1969 - accuracy: 0.9604 - val_loss: 0.1905 - val_accuracy: 0.9651
Epoc

### EfficientNet B4

In [15]:
enb4_wo_top = keras.applications.efficientnet.EfficientNetB4(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb4_wo_top.trainable = False

enb4_model = keras.models.Sequential()
enb4_model.add(enb4_wo_top)
enb4_model.add(keras.layers.Flatten())
enb4_model.add(keras.layers.Dense(128, activation='relu'))
enb4_model.add(keras.layers.Dense(64, activation='relu'))
enb4_model.add(keras.layers.Dense(8, activation='softmax'))

enb4_model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

enb4_model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test))


71686520/71686520 [==============================] - 0s 0us/step

Epoch 1/30


2023-08-08 16:14:13.795919: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/efficientnetb4/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


52/52 [==============================] - 39s 496ms/step - loss: 16.2970 - accuracy: 0.7723 - val_loss: 2.4699 - val_accuracy: 0.8170

Epoch 2/30

52/52 [==============================] - 22s 424ms/step - loss: 1.6939 - accuracy: 0.8611 - val_loss: 0.9919 - val_accuracy: 0.8668

Epoch 3/30

52/52 [==============================] - 18s 353ms/step - loss: 1.0390 - accuracy: 0.8907 - val_loss: 1.7910 - val_accuracy: 0.8053

Epoch 4/30

52/52 [==============================] - 22s 423ms/step - loss: 0.8329 - accuracy: 0.9070 - val_loss: 0.9539 - val_accuracy: 0.9139

Epoch 5/30

52/52 [==============================] - 22s 422ms/step - loss: 1.1016 - accuracy: 0.8979 - val_loss: 1.6362 - val_accuracy: 0.8931

Epoch 6/30

52/52 [==============================] - 18s 352ms/step - loss: 0.9206 - accuracy: 0.9046 - val_loss: 0.7742 - val_accuracy: 0.9130

Epoch 7/30

52/52 [==============================] - 22s 423ms/step - loss: 0.4585 - accuracy: 0.9330 - val_loss: 0.6695 - val_accuracy: 0.92

### EfficientNet B5

In [16]:
enb5_wo_top = keras.applications.efficientnet.EfficientNetB5(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb5_wo_top.trainable = False

enb5_model = keras.models.Sequential()
enb5_model.add(enb5_wo_top)
enb5_model.add(keras.layers.Flatten())
enb5_model.add(keras.layers.Dense(128, activation='relu'))
enb5_model.add(keras.layers.Dense(64, activation='relu'))
enb5_model.add(keras.layers.Dense(8, activation='softmax'))

enb5_model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

enb5_model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test))


115263384/115263384 [==============================] - 1s 0us/step

Epoch 1/30


2023-08-08 16:25:39.832937: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/efficientnetb5/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


52/52 [==============================] - 58s 794ms/step - loss: 14.6170 - accuracy: 0.7874 - val_loss: 3.3066 - val_accuracy: 0.8623

Epoch 2/30

52/52 [==============================] - 25s 492ms/step - loss: 3.8318 - accuracy: 0.8315 - val_loss: 3.0094 - val_accuracy: 0.8877

Epoch 3/30

52/52 [==============================] - 25s 491ms/step - loss: 3.3900 - accuracy: 0.8521 - val_loss: 5.2666 - val_accuracy: 0.8261

Epoch 4/30

52/52 [==============================] - 26s 509ms/step - loss: 1.6034 - accuracy: 0.8792 - val_loss: 1.1937 - val_accuracy: 0.8886

Epoch 5/30

52/52 [==============================] - 26s 508ms/step - loss: 1.3244 - accuracy: 0.8913 - val_loss: 0.9691 - val_accuracy: 0.9022

Epoch 6/30

52/52 [==============================] - 25s 490ms/step - loss: 0.6872 - accuracy: 0.9215 - val_loss: 0.8188 - val_accuracy: 0.9121

Epoch 7/30

52/52 [==============================] - 26s 509ms/step - loss: 0.6973 - accuracy: 0.9257 - val_loss: 0.7954 - val_accuracy: 0.91

### EfficientNet B6

In [17]:
enb6_wo_top = keras.applications.efficientnet.EfficientNetB6(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb6_wo_top.trainable = False

enb6_model = keras.models.Sequential()
enb6_model.add(enb6_wo_top)
enb6_model.add(keras.layers.Flatten())
enb6_model.add(keras.layers.Dense(128, activation='relu'))
enb6_model.add(keras.layers.Dense(64, activation='relu'))
enb6_model.add(keras.layers.Dense(8, activation='softmax'))

enb6_model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

enb6_model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test))


165234480/165234480 [==============================] - 1s 0us/step

Epoch 1/30


2023-08-08 16:39:21.370535: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/efficientnetb6/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


52/52 [==============================] - 66s 898ms/step - loss: 20.4183 - accuracy: 0.7627 - val_loss: 4.0538 - val_accuracy: 0.8659

Epoch 2/30

52/52 [==============================] - 41s 802ms/step - loss: 3.9529 - accuracy: 0.8339 - val_loss: 5.1934 - val_accuracy: 0.5806

Epoch 3/30

52/52 [==============================] - 41s 802ms/step - loss: 2.7704 - accuracy: 0.8460 - val_loss: 1.9560 - val_accuracy: 0.8832

Epoch 4/30

52/52 [==============================] - 33s 638ms/step - loss: 1.8828 - accuracy: 0.8581 - val_loss: 5.3015 - val_accuracy: 0.5761

Epoch 5/30

52/52 [==============================] - 41s 802ms/step - loss: 1.5618 - accuracy: 0.8702 - val_loss: 1.5069 - val_accuracy: 0.8560

Epoch 6/30

52/52 [==============================] - 41s 802ms/step - loss: 1.7237 - accuracy: 0.8798 - val_loss: 1.6337 - val_accuracy: 0.8786

Epoch 7/30

52/52 [==============================] - 41s 802ms/step - loss: 1.1548 - accuracy: 0.8998 - val_loss: 1.5990 - val_accuracy: 0.89

### EfficientNet B7

In [18]:
enb7_wo_top = keras.applications.efficientnet.EfficientNetB7(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

enb7_wo_top.trainable = False

enb7_model = keras.models.Sequential()
enb7_model.add(enb7_wo_top)
enb7_model.add(keras.layers.Flatten())
enb7_model.add(keras.layers.Dense(128, activation='relu'))
enb7_model.add(keras.layers.Dense(64, activation='relu'))
enb7_model.add(keras.layers.Dense(8, activation='softmax'))

enb7_model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

enb7_model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test))


258076736/258076736 [==============================] - 1s 0us/step

Epoch 1/30


2023-08-08 17:00:18.938491: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_4/efficientnetb7/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


52/52 [==============================] - 76s 1s/step - loss: 10.4503 - accuracy: 0.7766 - val_loss: 6.5107 - val_accuracy: 0.8632

Epoch 2/30

52/52 [==============================] - 48s 933ms/step - loss: 3.7033 - accuracy: 0.8484 - val_loss: 1.7107 - val_accuracy: 0.8723

Epoch 3/30

52/52 [==============================] - 44s 853ms/step - loss: 1.8217 - accuracy: 0.8545 - val_loss: 1.4632 - val_accuracy: 0.8587

Epoch 4/30

52/52 [==============================] - 48s 933ms/step - loss: 1.4227 - accuracy: 0.8738 - val_loss: 1.0049 - val_accuracy: 0.8859

Epoch 5/30

52/52 [==============================] - 48s 933ms/step - loss: 0.8374 - accuracy: 0.8859 - val_loss: 0.7162 - val_accuracy: 0.9049

Epoch 6/30

52/52 [==============================] - 48s 933ms/step - loss: 0.5495 - accuracy: 0.9064 - val_loss: 0.8654 - val_accuracy: 0.8949

Epoch 7/30

52/52 [==============================] - 48s 933ms/step - loss: 0.5023 - accuracy: 0.9058 - val_loss: 0.7524 - val_accuracy: 0.9022


In [19]:
resnet_wo_top = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(350, 350, 3))

resnet_wo_top.trainable = False

resnet_model = keras.models.Sequential()
resnet_model.add(resnet_wo_top)
resnet_model.add(keras.layers.Flatten())
resnet_model.add(keras.layers.Dense(128, activation='relu'))
resnet_model.add(keras.layers.Dense(64, activation='relu'))
resnet_model.add(keras.layers.Dense(8, activation='softmax'))

resnet_model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

resnet_model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test))


94765736/94765736 [==============================] - 1s 0us/step

Epoch 1/30

52/52 [==============================] - 19s 266ms/step - loss: 9.5755 - accuracy: 0.8110 - val_loss: 4.1062 - val_accuracy: 0.8940

Epoch 2/30

52/52 [==============================] - 10s 196ms/step - loss: 3.2696 - accuracy: 0.9004 - val_loss: 2.3007 - val_accuracy: 0.9221

Epoch 3/30

52/52 [==============================] - 11s 219ms/step - loss: 0.8234 - accuracy: 0.9457 - val_loss: 1.0823 - val_accuracy: 0.9429

Epoch 4/30

52/52 [==============================] - 10s 197ms/step - loss: 0.9841 - accuracy: 0.9499 - val_loss: 1.3779 - val_accuracy: 0.9167

Epoch 5/30

52/52 [==============================] - 10s 197ms/step - loss: 1.6150 - accuracy: 0.9372 - val_loss: 1.8411 - val_accuracy: 0.9239

Epoch 6/30

52/52 [==============================] - 10s 197ms/step - loss: 0.3091 - accuracy: 0.9746 - val_loss: 1.3336 - val_accuracy: 0.9447

Epoch 7/30

52/52 [==============================] - 10s 197ms/